# Sentinel 2 - Workflow
_Version: 0.4 &nbsp;
Author: T.Tewes_
> Verarbeitet Sentinel 2-Aufnahmen, die in einem Ordner liegen, anhand einer benuterdefinierten Auswahl;
> Zuschneiden auf eine AOI;
> NDVI-Berechnen, Statistische Auswertungen (folgen noch)

Download der Sentinel 2-Aufnahmen, z.B. über den EO-Explorer von CODE-DE. Es sollte auf eine geringe Wolkenbedeckung sowie eine Produktklassifikation L1C oder L2A geachtet werden. Alle heruntergeladenen Aufnahmen sollten in einem Ordner (siehe Pfade und Arbeitsbereich festlegen) gespeichert werden, wobei die original Ordnerstruktur der Sentinel 2 Szenen beibehalten werden sollte.

**SCL Korrektur funktioniert noch nicht**

## 1. Specifying the paths and working directories

In [1]:
import os

''' ---- Hier die Verzeichnisse angeben ---- '''
download_folder = r".\data\sentinel2-without-esri\download"
working_folder = r".\data\sentinel2-without-esri\working"
geotiff_folder = r".\data\sentinel2-without-esri\geotiff"
csv_folder = r".\data\sentinel2-without-esri\csv"
output_folder = r".\data\sentinel2-without-esri\output"
''' ----- Ende der Eingaben ---- '''

os.makedirs(download_folder, exist_ok=True)
os.makedirs(working_folder, exist_ok=True)
os.makedirs(geotiff_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

## 2. Download the datasets

Users can download raw Sentinel-2 datasets from https://explore.code-de.org/search

1. Signup/Login
2. Select Data Catalogue and Dataset
3. Draw polygon
4. Selected scenes
5. Download and place it under ./data/sentinel2-without-esri

## 3. Extract scenes

In [2]:
import os
import zipfile
from ipywidgets import SelectMultiple, Button, VBox, Output
from IPython.display import display

# List and sort scenes
scenes = sorted(f for f in os.listdir(download_folder) if f.endswith("SAFE.zip"))

# Selection widget
selection_widget = SelectMultiple(
    options=scenes,
    value=[scenes[0]],
    description="Scenes:",
    layout={'width': '80%', 'height': '150px'},
    style={'description_width': 'initial'}
)

# Create confirmation buttons
extract_button = Button(description="Extract Selected Scenes",
                        button_style='success',
                        layout={'width': '30%'})

# Output()
output = Output()

# Function to extract selected scenes
def extract_scenes(b):
    with output:
        output.clear_output()
        selected_scenes_for_extract = selection_widget.value

        if not selected_scenes_for_extract:
            print("No scenes selected for extraction.")
            return

        for scene in selected_scenes_for_extract:
            scene_path = os.path.join(download_folder, scene)
            scene_extract_folder = os.path.join(working_folder, scene.replace(".zip",""))
            
            if os.path.exists(scene_path):
                try:
                    with zipfile.ZipFile(scene_path, 'r') as zip_ref:
                        zip_ref.extractall(working_folder)
                    print(f"Extracted: {scene} to \n{scene_extract_folder}")
                except zipfile.BadZipFile:
                    print(f"Error: {scene} is not a valid zip file.")
            else:
                print(f"Scene not found: {scene}")
            print()
            
# Attach event handlers
extract_button.on_click(extract_scenes)

# Display the widgets
display(VBox([selection_widget, extract_button, output]))

## 4. Select scene from extracted for processing

In [3]:
# List and sort scenes only once
scenes = sorted(f for f in os.listdir(working_folder) if f.endswith(".SAFE"))

# Selection widget
selection_widget = SelectMultiple(
    options=scenes,
    value=[scenes[0]],
    description="Scenes:",
    layout={'width': '80%', 'height': '150px'},
    style={'description_width': 'initial'}
)

# Create confirmation buttons with adjusted width
process_button = Button(description="Select Scene",
                        button_style='success',
                        layout={'width': '30%'})

# Create output widget
output = Output()

selected_scenes_for_process = selection_widget.value

# Function for processing the selection
def process_selection(b):
    global selected_scenes_for_process
    selected_scenes_for_process = selection_widget.value
    with output:
        output.clear_output()
        if selected_scenes_for_process:
            print(f"Selected scenes ({len(selected_scenes_for_process)}):")
            print('\n'.join([f"- {scene}" for scene in selected_scenes_for_process]))
        else:
            print("No scenes selected.")

# Attach event handlers
process_button.on_click(process_selection)

# Display the widgets
display(VBox([selection_widget, process_button, output]))

## Process

In [4]:
import os
import numpy as np
import rasterio
import logging
import geopandas as gpd

# Constants
NDVI_BANDS = {"NIR": "B08", "RED": "B04"}
RESOLUTION_DIRS = ["R10m", "R20m", "R60m"]

def calculate_ndvi(img_data_dir, image_filenames_list, aoi_shapefile=None):
    try:
        # Locate required bands
        nir_filename = next((f for f in image_filenames_list if NDVI_BANDS["NIR"] in f), None)
        red_filename = next((f for f in image_filenames_list if NDVI_BANDS["RED"] in f), None)

        if not nir_filename or not red_filename:
            raise FileNotFoundError("NIR or RED band files not found in the provided list.")

        # Construct full file paths
        nir_band_path = os.path.join(img_data_dir, nir_filename)
        red_band_path = os.path.join(img_data_dir, red_filename)

        # Read bands
        with rasterio.open(nir_band_path) as nir_src:
            nir_band = nir_src.read(1).astype("float32")
            transform = nir_src.transform
            crs = nir_src.crs

        with rasterio.open(red_band_path) as red_src:
            red_band = red_src.read(1).astype("float32")

        # Calculate NDVI
        with np.errstate(divide="ignore", invalid="ignore"):
            ndvi = (nir_band - red_band) / (nir_band + red_band)
            ndvi = np.nan_to_num(ndvi, nan=0.0)  # Replace NaN with 0

        return ndvi, transform, crs

    except Exception as e:
        logging.error(f"Error in NDVI calculation: {e}")
        return None, None, None
    
def save_raster(output_path, data, transform, crs):
    try:
        with rasterio.open(
            output_path,
            "w",
            driver="GTiff",
            height=data.shape[0],
            width=data.shape[1],
            count=1,
            dtype=data.dtype,
            crs=crs,
            transform=transform,
        ) as dst:
            dst.write(data, 1)
        logging.info(f"Raster saved to {output_path}")
    except Exception as e:
        logging.error(f"Error saving raster: {e}")

In [5]:
def process_scene(selected_scenes, output_folder, index="NDVI", aoi_shapefile=None):
    os.makedirs(output_folder, exist_ok=True)

    for scene in selected_scenes:
        try:
            # Define paths for the scene
            scene_extract_folder = os.path.join(working_folder, scene)
            granule_dir = os.path.join(scene_extract_folder, "GRANULE")

            # Validate granule directory
            if not os.path.exists(granule_dir) or not os.listdir(granule_dir):
                logging.error(f"Granule directory missing or empty for scene {scene}")
                continue

            granule_subdirs = os.listdir(granule_dir)
            granule_subdir = granule_subdirs[0]
            img_data_dir = os.path.join(granule_dir, granule_subdir, "IMG_DATA")

            # Prioritize R10m resolution if available
            for res_dir in RESOLUTION_DIRS:
                if res_dir in os.listdir(img_data_dir):
                    img_data_dir = os.path.join(img_data_dir, res_dir)
                    break

            # Validate image data directory
            if not os.path.exists(img_data_dir) or not os.listdir(img_data_dir):
                logging.error(f"Image data directory missing or empty for scene {scene}")
                continue

            image_filenames_list = os.listdir(img_data_dir)

            # Calculate the specified index (e.g., NDVI)
            if index == "NDVI":
                ndvi, transform, crs = calculate_ndvi(img_data_dir, image_filenames_list)
                if ndvi is not None:
                    output_path = os.path.join(output_folder, f"{scene}_NDVI.tif")
                    save_raster(output_path, ndvi, transform, crs)
        except Exception as e:
            logging.error(f"Error processing scene {scene}: {e}")

if __name__ == '__main__':
    # Export complete scene
    process_scene(
        selected_scenes=selected_scenes_for_process,
        output_folder=geotiff_folder,
        index="NDVI",
        aoi_shapefile=None
        )
    
    # # Export clipped scene using shapefile
    # kn_shapefile = "./shapefiles/kn_boundary.shp"
    # process_scene(
    #     selected_scenes=selected_scenes_for_process,
    #     output_folder=geotiff_folder,
    #     index="NDVI",
    #     aoi_shapefile=kn_shapefile,
    #     )